## TEST NEW

In [1]:
# lazy import
import sys
sys.path.append("/home/deren/hacks/hogtie/")

In [2]:
import hogtie
hogtie.set_loglevel("INFO")

In [3]:
import subprocess
import toytree
import numpy as np

### test data

In [4]:
TREE = toytree.rtree.baltree(ntips=10, treeheight=10)
DATA = np.array([0, 0, 0, 1, 1, 1, 1, 1, 0, 1]).astype(float)

### Fit with hogtie

In [5]:
model = hogtie.BinaryStateModel(TREE, DATA)
model.optimize()

06:12 | INFO    | optimize        | {'alpha': 0.102567, 'beta': 0.0585, 'Lik': -0.002575, 'negLogLik': 5.96, 'convergence': True}


In [11]:
# inferred parameters
model.alpha, model.beta

(0.10256728309648974, 0.058500294971227704)

In [12]:
# negative log-likelihood of the data given the model
model.log_lik

5.96

In [13]:
# draw inferred states
model.draw_states();

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="400.0px" height="300.0px" viewBox="0 0 400.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="td2fc0db4035c4d98ab0eef550004d77d"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 r0 r1 r2 r3 r4 r5 r6 r7 r8 r9

In [14]:
# show relative likelihoods
model.tree.get_node_values("likelihood", 1, 1)

array([[0.48180651, 0.51819349],
       [0.30632986, 0.69367014],
       [0.65676495, 0.34323505],
       [0.58699789, 0.41300211],
       [0.11309315, 0.88690685],
       [0.16264176, 0.83735824],
       [0.94784347, 0.05215653],
       [0.05444848, 0.94555152],
       [0.97724313, 0.02275687],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ]])

### TEST WITH R

In [15]:

out = subprocess.run([
    "Rscript", 
    "-e", "library(ape)",
    "-e", f"tree <- read.tree(text='{TREE.newick}')",
    "-e", f"data <- c{tuple(DATA[::-1])}",
    "-e", "object <- ace(data, tree, marginal=TRUE, type='discrete', model='ARD', use.expm=TRUE)",
    "-e", "print(object)",
    "-e", "print(object$lik.anc)",
    ], capture_output=True,
)

print(out.stdout.decode())


    Ancestral Character Estimation

Call: ace(x = data, phy = tree, type = "discrete", model = "ARD", use.expm = TRUE, 
    marginal = TRUE)

    Log-likelihood: -5.268686 

Rate index matrix:
  0 1
0 . 2
1 1 .

Parameter estimates:
 rate index estimate std-err
          1   0.0585  0.0430
          2   0.1026  0.0695

Scaled likelihoods at the root (type '...$lik.anc' to get them for all nodes):
        0         1 
0.4818146 0.5181854 
               0          1
 [1,] 0.48181457 0.51818543
 [2,] 0.30632514 0.69367486
 [3,] 0.58701036 0.41298964
 [4,] 0.11308282 0.88691718
 [5,] 0.05444339 0.94555661
 [6,] 0.65678039 0.34321961
 [7,] 0.16262925 0.83737075
 [8,] 0.94785412 0.05214588
 [9,] 0.97724780 0.02275220

